In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import chart_studio.plotly as ply
import os
from scipy import interpolate as interp
from scipy.signal import spectrogram, firwin, filtfilt, hilbert
from utilities.signal_analysis_methods import get_harmonic_envelope, get_total_envelope
import utm

In [ ]:
mag_path = 'data/mag_engi_7-3-25.csv'
gps_path = 'data/gps_engi_7-3-25.csv'

# magnetic coil data acquisition sampling rate
sample_rate = int(2e3) # in [Hz]
driving_frequency = 100 # in [Hz]
mag_data = pd.read_csv(mag_path)
mag_times = mag_data['Times (s)'].to_numpy()[:sample_rate*500]
mag_ch1_voltages = mag_data['Secondary Ch 1 Voltage (V)'].to_numpy()[:sample_rate*500]
mag_ch2_voltages = mag_data['Secondary Ch 2 Voltage (V)'].to_numpy()[:sample_rate*500]
mag_primary_voltages = mag_data['Primary Voltage (V)'].to_numpy()[:sample_rate*500]
# AC voltages computed by subtracting means from raw voltages
mag_ch1_ac_voltages = mag_ch1_voltages-np.mean(mag_ch1_voltages)
mag_ch2_ac_voltages = mag_ch2_voltages-np.mean(mag_ch2_voltages)
mag_primary_ac_voltages = mag_primary_voltages-np.mean(mag_primary_voltages)
# RMS voltages computed by subtracting means from the RMS of the AC voltages
num_rms_bins = int(len(mag_ch1_ac_voltages)/sample_rate)
mag_ch1_rms_voltages = [np.sqrt(np.mean(np.square(mag_ch1_ac_voltages[i*sample_rate:(i+1)*sample_rate]))).tolist() for i in range(num_rms_bins)]
mag_ch2_rms_voltages = [np.sqrt(np.mean(np.square(mag_ch2_ac_voltages[i*sample_rate:(i+1)*sample_rate]))).tolist() for i in range(num_rms_bins)]
mag_ch1_rms_voltages = mag_ch1_rms_voltages - np.mean(mag_ch1_rms_voltages)
mag_ch2_rms_voltages = mag_ch2_rms_voltages - np.mean(mag_ch2_rms_voltages)

gps_data = pd.read_csv(gps_path)
gps_times = gps_data['Times (s)'].to_numpy()[:500]
gps_lat = gps_data['Latitude'].to_numpy()[:500]
gps_lon = gps_data['Longitude'].to_numpy()[:500]
# Conversion from GPS global latitude and longitude to UTM latitude and longitude (in [m])
utm_lat, utm_lon, zone_num, zone_letter = utm.from_latlon(gps_lat, gps_lon)

Interpolation of RMS Voltages to GPS Coordinates

In [ ]:
# Interpolate rms mag data to gps coords
mag_ch1_voltages_interp = interp.interp1d(mag_times[::sample_rate], mag_ch1_rms_voltages, kind='cubic', fill_value='extrapolate')
mag_ch2_voltages_interp = interp.interp1d(mag_times[::sample_rate], mag_ch2_rms_voltages, kind='cubic', fill_value='extrapolate')
mag_ch1_voltages_at_gps_coords = mag_ch1_voltages_interp(gps_times)
mag_ch2_voltages_at_gps_coords = mag_ch2_voltages_interp(gps_times)

# Interpolate fundamental frequency mag data to gps coords
mag_ch1_fundamental_envelope = get_harmonic_envelope(mag_ch1_ac_voltages, sample_rate, mag_primary_ac_voltages, driving_frequency, 1)[2]
mag_ch1_fundamental_envelope = mag_ch1_fundamental_envelope - np.mean(mag_ch1_fundamental_envelope)
mag_ch1_fundamental_envelope_interp = interp.interp1d(mag_times, mag_ch1_fundamental_envelope, kind='cubic', fill_value='extrapolate')
mag_ch1_fundamental_envelope_at_gps_coords = mag_ch1_fundamental_envelope_interp(gps_times)
mag_ch2_fundamental_envelope = get_harmonic_envelope(mag_ch2_ac_voltages, sample_rate, mag_primary_ac_voltages, driving_frequency, 1)[2]
mag_ch2_fundamental_envelope = mag_ch2_fundamental_envelope - np.mean(mag_ch2_fundamental_envelope)
mag_ch2_fundamental_envelope_interp = interp.interp1d(mag_times, mag_ch2_fundamental_envelope, kind='cubic', fill_value='extrapolate')
mag_ch2_fundamental_envelope_at_gps_coords = mag_ch2_fundamental_envelope_interp(gps_times)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x= mag_times[::sample_rate], 
    y= mag_ch1_rms_voltages, 
    mode='lines', 
    name='Ch 1 RMS Voltages',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times[1000:-1000],
    y= mag_ch1_fundamental_envelope[1000:-1000]-np.mean(mag_ch1_fundamental_envelope[1000:-1000]),   
    mode='lines', 
    name='Ch 1 Fundamental Envelope',
    line=dict(color='royalblue', width=2)
))
fig.update_layout(
    title='Ch 1 RMS Voltage and Fundamental Envelope vs. Time',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (V)',
    template='plotly_dark',
    hovermode='x unified',
)
fig.show()

Cutting out the turns

In [ ]:
slice_indices = [(27,47), (65,85), (95,115), (130,150), (157,177), (185,205), (214,234), (235,245), \
                 (245,365), (285,305), (320,340), (357,377), (395,415), (435,455), (475,495)]

mag_ch1_fundamental_envelope_sliced = np.copy(mag_ch1_fundamental_envelope)
mag_ch1_fundamental_envelope_at_gps_coords_sliced = np.copy(mag_ch1_fundamental_envelope_at_gps_coords)
mag_ch2_fundamental_envelope_sliced = np.copy(mag_ch2_fundamental_envelope)
mag_ch2_fundamental_envelope_at_gps_coords_sliced = np.copy(mag_ch2_fundamental_envelope_at_gps_coords)

for i,indices in enumerate(slice_indices):
    mag_ch1_fundamental_envelope_sliced[int(indices[0]*sample_rate):int(indices[1]*sample_rate)] = 0
    mag_ch1_fundamental_envelope_at_gps_coords_sliced[indices[0]:indices[1]] = 0
    #mag_ch2_fundamental_envelope_sliced[int(indices[0]*sample_rate):int(indices[1]*sample_rate)] = 0
    #mag_ch2_fundamental_envelope_at_gps_coords_sliced[indices[0]:indices[1]] = 0
    
mag_ch2_fundamental_envelope_sliced[int(165*sample_rate):int(240*sample_rate)] = 0
mag_ch2_fundamental_envelope_at_gps_coords_sliced[165:240] = 0
mag_ch2_fundamental_envelope_sliced[int(445*sample_rate):int(490*sample_rate)] = 0
mag_ch2_fundamental_envelope_at_gps_coords_sliced[445:490] = 0

fig = go.Figure()
fig.add_trace(go.Scatter(
    x= mag_times[1000:-1000],
    y= mag_ch1_fundamental_envelope_sliced[1000:-1000],   
    mode='lines', 
    name='Ch 1 Fundamental Envelope (Turns Excluded)',
    line=dict(color='royalblue', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times[1000:-1000],
    y= mag_ch1_fundamental_envelope[1000:-1000],   
    mode='lines', 
    name='Ch 1 Fundamental Envelope',
    line=dict(color='rebeccapurple', width=2)
))
fig.update_layout(
    title='Ch 1 Fundamental Envelope vs. Time',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (V)',
    template='plotly_dark',
    hovermode='x unified',
)
fig.show()

mag_map = plt.scatter(x=utm_lat[10:-10]%1000, y=utm_lon[10:-10]%1000, c=mag_ch1_fundamental_envelope_at_gps_coords_sliced[10:-10], cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 Fundamental Envelope Map (Turns Excluded)')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_map = plt.scatter(x=utm_lat[10:-10]%1000, y=utm_lon[10:-10]%1000, c=mag_ch1_fundamental_envelope_at_gps_coords[10:-10], cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 Fundamental Envelope Map')
plt.axis('equal')
plt.grid(True)
plt.show()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x= mag_times[1000:-1000],
    y= mag_ch2_fundamental_envelope_sliced[1000:-1000],   
    mode='lines', 
    name='Ch 1 Fundamental Envelope (Turns Excluded)',
    line=dict(color='royalblue', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times[1000:-1000],
    y= mag_ch2_fundamental_envelope[1000:-1000],   
    mode='lines', 
    name='Ch 1 Fundamental Envelope',
    line=dict(color='rebeccapurple', width=2)
))
fig.update_layout(
    title='Ch 2 Fundamental Envelope vs. Time',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (V)',
    template='plotly_dark',
    hovermode='x unified',
)
fig.show()

mag_map = plt.scatter(x=utm_lat[10:-10]%1000, y=utm_lon[10:-10]%1000, c=mag_ch2_fundamental_envelope_at_gps_coords_sliced[10:-10], cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 2 Fundamental Envelope Map (Turns Excluded)')
plt.axis('equal')
plt.grid(True)
plt.show()

# mag_map = plt.scatter(x=utm_lat[10:-10]%1000, y=utm_lon[10:-10]%1000, c=mag_ch2_fundamental_envelope_at_gps_coords[10:-10], cmap='viridis')
# plt.colorbar(mag_map, label='Voltage (V)')
# plt.xlabel('Latitude (m)')
# plt.ylabel('Longitude (m)')
# plt.title('Ch 2 Fundamental Envelope Map')
# plt.axis('equal')
# plt.grid(True)
# plt.show()

Mapping of Secondary Coil Voltages and Complex Field Phase

In [ ]:
mag_map = plt.scatter(x=utm_lat[10:-10]%1000, y=utm_lon[10:-10]%1000, c=mag_ch1_fundamental_envelope_at_gps_coords[10:-10], cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 Fundamental Envelope Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_map = plt.scatter(x=utm_lat[10:-10]%1000, y=utm_lon[10:-10]%1000, c=mag_ch1_fundamental_envelope_at_gps_coords[10:-10], cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 Fundamental Envelope Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_ch1_voltages_at_gps_coords, cmap='viridis')
plt.colorbar(mag_map, label='Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 RMS Voltage Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_ch2_voltages_at_gps_coords, cmap='viridis')
plt.colorbar(mag_map, label='Ch 2 Voltage (V)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 2 Voltage Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_field_phase = np.arctan2(mag_ch2_voltages_at_gps_coords, mag_ch1_voltages_at_gps_coords)
mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_field_phase, cmap='viridis')
plt.colorbar(mag_map, label='arctan(Ch2/Ch1)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Phase Map')
plt.axis('equal')
plt.grid(True)
plt.show()

Mapping of Secondary Coil Fundamental and Third Harmonic Frequency Components and Complex Field Phase (at the fundamental)

In [ ]:
frequencies, times, Sxx = spectrogram(mag_ch1_ac_voltages, fs=sample_rate, nperseg=1024)
mag_ch1_fundamental_interp = interp.interp1d(times, Sxx[10], kind='cubic', fill_value='extrapolate')
mag_ch1_fundamental_at_gps_coords = mag_ch1_fundamental_interp(gps_times)
mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_ch1_fundamental_at_gps_coords, cmap='magma')
plt.colorbar(mag_map, label='Power')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 100 Hz Fundamental Map')
plt.axis('equal')
plt.grid(True)
plt.show()

frequencies, times, Sxx = spectrogram(mag_ch2_ac_voltages, fs=sample_rate, nperseg=1024)
mag_ch2_fundamental_interp = interp.interp1d(times, Sxx[10], kind='cubic', fill_value='extrapolate')
mag_ch2_fundamental_at_gps_coords = mag_ch2_fundamental_interp(gps_times)
mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_ch2_fundamental_at_gps_coords, cmap='magma')
plt.colorbar(mag_map, label='Power')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 2 100 Hz Fundamental Map')
plt.axis('equal')
plt.grid(True)
plt.show()

frequencies, times, Sxx = spectrogram(mag_ch1_ac_voltages, fs=sample_rate, nperseg=1024)
mag_ch1_3harmonic_interp = interp.interp1d(times, Sxx[30], kind='cubic', fill_value='extrapolate')
mag_ch1_3harmonic_at_gps_coords = mag_ch1_3harmonic_interp(gps_times)
mag_map = plt.scatter(x=utm_lat%1000, y=utm_lon%1000, c=mag_ch1_fundamental_at_gps_coords, cmap='magma')
plt.colorbar(mag_map, label='Power')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 1 300 Hz Harmonic Map')
plt.axis('equal')
plt.grid(True)
plt.show()

frequencies, times, Sxx = spectrogram(mag_ch2_ac_voltages, fs=sample_rate, nperseg=1024)
mag_ch2_3harmonic_interp = interp.interp1d(times, Sxx[30], kind='cubic', fill_value='magma')
mag_ch2_3harmonic_at_gps_coords = mag_ch2_3harmonic_interp(gps_times[1:])
mag_map = plt.scatter(x=utm_lat[1:]%1000, y=utm_lon[1:]%1000, c=mag_ch2_3harmonic_at_gps_coords, cmap='viridis')
plt.colorbar(mag_map, label='Power')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Ch 2 300 Hz Harmonic Map')
plt.axis('equal')
plt.grid(True)
plt.show()

mag_field_phase = np.arctan2(mag_ch2_3harmonic_at_gps_coords, mag_ch1_3harmonic_at_gps_coords[1:])
mag_map = plt.scatter(x=utm_lat[1:]%1000, y=utm_lon[1:]%1000, c=mag_field_phase, cmap='magma')
plt.colorbar(mag_map, label='arctan(Ch2/Ch1)')
plt.xlabel('Latitude (m)')
plt.ylabel('Longitude (m)')
plt.title('Phase Map of Third Harmonic')
plt.axis('equal')
plt.grid(True)
plt.show()


In [ ]:
logname = "arcgis_data.csv"
path = os.path.expanduser('data/'+logname)
logfile = open(path, "w")
logfile.write("Times (s),GPS Lat,GPS Lon,Ch 1 Voltage (V),Ch 2 Voltage (V),Phase\n")
for i in range(len(gps_times)):
        logfile.write(f"{gps_times[i]},{gps_lat[i]},{gps_lon[i]},\
                        {mag_ch1_voltages_at_gps_coords[i]:.7f},{mag_ch2_voltages_at_gps_coords[i]:.7f},{mag_field_phase[i]:.7f}\n")
logfile.close()